### CSV and Excel Parsing - Structured Data

In [7]:
import pandas as pd
import os

In [ ]:
os.makedirs("data/structured_files2", exist_ok=True)

# Create sample data
data = {
    'Product': ['Laptop', 'Mouse', 'Keyboard', 'Monitor', 'Webcam'],
    'Category': ['Electronics', 'Accessories', 'Accessories', 'Electronics', 'Electronics'],
    'Price': [999.99, 29.99, 79.99, 299.99, 89.99],
    'Stock': [50, 200, 150, 75, 100],
    'Description': [
        'High-performance laptop with 16GB RAM and 512GB SSD',
        'Wireless optical mouse with ergonomic design',
        'Mechanical keyboard with RGB backlighting',
        '27-inch 4K monitor with HDR support',
        '1080p webcam with noise cancellation'
    ]
}

# Create DataFrame
df = pd.DataFrame(data)

df.to_csv("data/structured_files2/products.csv", index=False)


In [10]:
# Save as Excel with multiple sheets
with pd.ExcelWriter('data/structured_files2/inventory.xlsx') as writer:
    df.to_excel(writer, sheet_name='Products', index=False)
    
    # Add another sheet
    summary_data = {
        'Category': ['Electronics', 'Accessories'],
        'Total_Items': [3, 2],
        'Total_Value': [1389.97, 109.98]
    }
    pd.DataFrame(summary_data).to_excel(writer, sheet_name='Summary', index=False)

### CSV Processing

In [11]:
from langchain_community.document_loaders import CSVLoader
from langchain_community.document_loaders import UnstructuredCSVLoader


In [ ]:
# Method 1: CSVLoader - Each row becomes a document
print("CSVLoader - Row-based Documents")

csv_loader = CSVLoader(
    file_path='data/structured_files/products.csv',
    encoding='utf-8',
    csv_args={
        'delimiter': ',',
        'quotechar': '"',
    }
)

csv_docs = csv_loader.load()
print(f"Loaded {len(csv_docs)} documents (one per row)")
print("\nFirst document:")
print(f"Content: {csv_docs[0].page_content}")
print(f"Meta {csv_docs[0].metadata}")


In [18]:
from langchain_core.documents import Document
# Method 2: Custom CSV processing for better control
print("\n2. Custom CSV Processing")


def process_csv_intelligently(filepath: str) -> list[Document]:
    """Process CSV with intelligent document creation"""
    df = pd.read_csv(filepath)
    documents = []

    # Strategy 1: One document per row with structured content
    for idx, row in df.iterrows():
        # Create structured content
        content = f"""Product Information:
        Name: {row['Product']}
        Category: {row['Category']}
        Price: ${row['Price']}
        Stock: {row['Stock']} units
        Description: {row['Description']}"""

        # Create document with rich metadata
        doc = Document(
            page_content=content,
            metadata={
                'source': filepath,
                'row_index': idx,
                'product_name': row['Product'],
                'category': row['Category'],
                'price': row['Price'],
                'data_type': 'product_info'
            }
        )
        documents.append(doc)
    return documents

process_csv_intelligently("data/structured_files2/products.csv")



2. Custom CSV Processing


[Document(metadata={'source': 'data/structured_files2/products.csv', 'row_index': 0, 'product_name': 'Laptop', 'category': 'Electronics', 'price': 999.99, 'data_type': 'product_info'}, page_content='Product Information:\n        Name: Laptop\n        Category: Electronics\n        Price: $999.99\n        Stock: 50 units\n        Description: High-performance laptop with 16GB RAM and 512GB SSD'),
 Document(metadata={'source': 'data/structured_files2/products.csv', 'row_index': 1, 'product_name': 'Mouse', 'category': 'Accessories', 'price': 29.99, 'data_type': 'product_info'}, page_content='Product Information:\n        Name: Mouse\n        Category: Accessories\n        Price: $29.99\n        Stock: 200 units\n        Description: Wireless optical mouse with ergonomic design'),
 Document(metadata={'source': 'data/structured_files2/products.csv', 'row_index': 2, 'product_name': 'Keyboard', 'category': 'Accessories', 'price': 79.99, 'data_type': 'product_info'}, page_content='Product Info